In [1]:
import pandas as pd

student_data = pd.read_csv("ML - Curricular Analytics - PIDM ONLY & Fixed Repeat IND.csv")
grades = pd.read_csv("parsed_grades.csv")
student_data.shape

C:\Users\judem\AppData\Local\Temp\ipykernel_3348\2249383029.py:3: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  student_data = pd.read_csv("ML - Curricular Analytics - PIDM ONLY & Fixed Repeat IND.csv")


(2677409, 33)

In [2]:
#Add Numeric GPA Column

grade_mapping = dict(grades[['Code', 'Quality Points']].values)

student_data = student_data.drop(student_data[(student_data.FINAL_GRADE == 'W') | 
                                              (student_data.FINAL_GRADE == 'S') | 
                                              (student_data.FINAL_GRADE == 'U') | 
                                              (student_data.FINAL_GRADE.isna()) |
                                              (student_data.FINAL_GRADE == '')].index)

student_data['numeric_gpa'] = student_data.FINAL_GRADE.map(grade_mapping)
student_data['numeric_gpa'] = student_data['numeric_gpa'].astype(float)

student_data.shape

(2510910, 34)

In [3]:
#Final and Semester GPAs (Assuming all classes are equal credits)

student_final_gpa = student_data.groupby('Pidm')['numeric_gpa'].mean().reset_index()

student_semester_gpa = student_data.groupby(['Pidm', 'Term'])['numeric_gpa'].mean().reset_index()

# print(student_final_gpa)
# print(student_semester_gpa)
student_data.shape

(2510910, 34)

In [4]:
#Student Classes per Semester (As an array of strings)

student_data['class'] = (student_data['SUBJ'] + student_data['CRSE_NUMB']).astype(str)
semester_classes = student_data.groupby(['Pidm', 'Term']).agg({
    'FINAL_GRADE': list,
    'class': list
}).reset_index()

# semester_classes.head()
student_data.shape

(2510910, 35)

In [5]:
student_gpa_map = dict(student_final_gpa[['Pidm', 'numeric_gpa']].values)
student_data['Final GPA'] = student_data.Pidm.map(student_gpa_map)

student_data.drop(['CRN', 'SUBJ', 'CRSE_NUMB', 'REPEAT_IND', 'FINAL_GRADE', 'numeric_gpa', 'class'], axis=1, inplace=True)
# student_data.head()
student_data.shape
# display(student_data[student_data["Pidm"] == 3963])

(2510910, 29)

In [6]:
student_data = student_data.groupby(['Pidm', 'Term']).agg({ 
    'Admit_Code': 'first', 
    'Admit_Level': 'first', 
    'Admit_College': 'first', 
    'Admit_Major_Code': 'first', 
    'Major_Desc': 'first', 
    'Trump_Race': 'first', 
    'Trump_Race_Desc': 'first', 
    'MULTI': 'first', 
    'Race': 'first', 
    'NEW_ETHNICITY': 'first', 
    'GENDER_Code': 'first', 
    'GENDER': 'first', 
    'CITZ_IND': 'first', 
    'CITZ_CODE': 'first', 
    'CITZ_DESC': 'first', 
    'Final_GPA': 'first', 
    'ACTE': 'first', 
    'ACTM': 'first', 
    'ACTR': 'first', 
    'ACTS': 'first', 
    'EACT': 'first', 
    'SAT-ERW': 'first', 
    'SATM': 'first', 
    'SAT_TOTAL': 'first', 
    'Final GPA': 'first'
}).reset_index()

student_data.shape



(663750, 27)

In [7]:
student_data['Semester GPA'] = student_data.merge(student_semester_gpa[['Pidm', 'Term', 'numeric_gpa']], on=['Pidm', 'Term'], how='left')['numeric_gpa']

student_data = student_data.merge(semester_classes[['Pidm', 'Term', 'FINAL_GRADE', 'class']], on=['Pidm', 'Term'], how='left')
student_data.rename(columns={'Final_GPA':'HS GPA', 'Term':'Semester','FINAL_GRADE':'Semester Grades', 'class':'Classes'}, inplace=True)



student_data.head()
# student_data.shape

,Pidm,Semester,Admit_Code,Admit_Level,Admit_College,Admit_Major_Code,Major_Desc,Trump_Race,Trump_Race_Desc,MULTI,...,ACTR,ACTS,EACT,SAT-ERW,SATM,SAT_TOTAL,Final GPA,Semester GPA,Semester Grades,Classes
0,285,202101.0,FS,UG,SA,BGS,General Studies,W,White,None,...,-,-,-,-,-,-,3.750000,3.75,"[A, A-, B+, A]","[LIS3361, LIS3353, COP2030, CIS4510]"
1,432,201908.0,FS,UG,SA,BGS,General Studies,B,Black or African American,Multi-Race,...,-,-,-,-,-,-,0.000000,0.00,"[IF, F, F]","[MUS4930, CCJ3117, IDS4934]"
2,705,202208.0,SB,UG,NR,PNR,Pre-Nursing,W,White,None,...,36,29,30,-,-,-,4.000000,4.00,"[A, A+, A, A, A, A]","[MCB2000, MCB2000L, HUN2201, BSC2085, BSC2085L..."
3,794,201508.0,SB,UG,SA,PNR,Pre-Nursing,W,White,None,...,25,22,25,-,-,-,3.977241,4.00,"[A+, A+, A+]","[NUR3805, NUR3078, NUR4895]"
4,794,201601.0,SB,UG,SA,PNR,Pre-Nursing,W,White,None,...,25,22,25,-,-,-,3.977241,4.00,"[A+, A+]","[NUR4828C, NUR4128]"


In [8]:
student_data = student_data.astype(str)
student_data['Pidm'] = student_data['Pidm'].astype(int)
student_data['Final GPA'] = student_data['Final GPA'].astype(float).round(2)
student_data['Semester GPA'] = student_data['Semester GPA'].astype(float).round(2)
student_data['Semester'] = student_data['Semester'].astype(float).astype(int).astype(str)
student_data.to_csv("formatted_data.csv", index=False)

In [9]:
semester_classes.head()

,Pidm,Term,FINAL_GRADE,class
0,285,202101.0,"[A, A-, B+, A]","[LIS3361, LIS3353, COP2030, CIS4510]"
1,432,201908.0,"[IF, F, F]","[MUS4930, CCJ3117, IDS4934]"
2,705,202208.0,"[A, A+, A, A, A, A]","[MCB2000, MCB2000L, HUN2201, BSC2085, BSC2085L..."
3,794,201508.0,"[A+, A+, A+]","[NUR3805, NUR3078, NUR4895]"
4,794,201601.0,"[A+, A+]","[NUR4828C, NUR4128]"
